In [12]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'Iris.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [13]:
## changing labels to numbers
iris['Species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1,
                                     np.where(iris['Species']== 'Iris-versicolor', 2, 3))

In [33]:
## defining inputs and target variables
x = iris.drop(columns = ['Id', 'Species', 'Species_numb'], axis = 1)
y = iris['Species_numb']

## splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [34]:
## defining the scaler
scaler = MinMaxScaler()

## scaling the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

### One v One (Logistic Regression and Decision Tree)
***

In [35]:
## building models
one_v_one_logit = OneVsOneClassifier(estimator = LogisticRegression()).fit(x_train, y_train)
one_v_one_dc = OneVsOneClassifier(estimator = DecisionTreeClassifier()).fit(x_train, y_train)

## predicting on the testing dataset
one_v_one_logit_preds = one_v_one_logit.predict(x_test)
one_v_one_dc_preds = one_v_one_dc.predict(x_test)

## creating classification model LOGIT
print('Logistic Regression')
print(classification_report(y_test, one_v_one_logit_preds))

## creating confusion matrix LOGIT
print(confusion_matrix(y_test, one_v_one_logit_preds))

## creating classification report DC
print('Decision Tree')
print(classification_report(y_test, one_v_one_dc_preds))

## creating confusion matrix DC
print(confusion_matrix(y_test, one_v_one_dc_preds))

Logistic Regression
              precision    recall  f1-score   support

           1       0.75      0.90      0.82        10
           2       0.88      0.70      0.78        10
           3       1.00      1.00      1.00        10

    accuracy                           0.87        30
   macro avg       0.88      0.87      0.87        30
weighted avg       0.88      0.87      0.87        30

[[ 9  1  0]
 [ 3  7  0]
 [ 0  0 10]]
Decision Tree
              precision    recall  f1-score   support

           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

[[ 9  1  0]
 [ 1  9  0]
 [ 0  0 10]]


based on these results, the decision tree (93% F1) has better performance compared to logistic regression (87% f1 score) and should be selected for further modeling purposes.